In [1]:
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [10]:

BASE_DIR = Path('../data/processed_data')

# Sequence length  (match your LSTM window size)
WINDOW_LEN = 50

# Pitch range for piano-roll
PITCH_START = 21
PITCH_END   = 108
NUM_PITCHES = PITCH_END - PITCH_START + 1  # 88

### Define Piano-Roll Conversion

In [11]:
def seq_to_pianoroll(seq,
                     pitch_start=PITCH_START,
                     num_pitches=NUM_PITCHES):
    """
    seq: np.array of shape (WINDOW_LEN, 3), where seq[:,0] = pitch.
    Returns a binary piano-roll: shape (1, num_pitches, WINDOW_LEN).
    """
    pr = np.zeros((num_pitches, seq.shape[0]), dtype=np.float32)
    for t, note in enumerate(seq):
        p = int(note[0])
        pr[p - pitch_start, t] = 1.0
    return pr[np.newaxis, :, :]  # add channel dimension

### Load LSTM Windows & Build CNN Features

In [12]:
# Load LSTM windows + labels
with open(BASE_DIR/'lstm_train.pkl','rb') as f:
    X_train_l, y_train = pickle.load(f)
with open(BASE_DIR/'lstm_dev.pkl','rb') as f:
    X_dev_l,   y_dev   = pickle.load(f)
with open(BASE_DIR/'lstm_test.pkl','rb') as f:
    X_test_l,  y_test  = pickle.load(f)

# Convert each window to a piano-roll
X_train_c = np.stack([seq_to_pianoroll(seq) for seq in X_train_l], axis=0)
X_dev_c   = np.stack([seq_to_pianoroll(seq) for seq in X_dev_l],   axis=0)
X_test_c  = np.stack([seq_to_pianoroll(seq) for seq in X_test_l],  axis=0)

print("LSTM shapes:", X_train_l.shape, X_dev_l.shape, X_test_l.shape)
print("CNN shapes: ", X_train_c.shape, X_dev_c.shape, X_test_c.shape)
print("Labels train/dev/test:", y_train.shape, y_dev.shape, y_test.shape)


LSTM shapes: (480044, 50, 3) (41034, 50, 3) (39295, 50, 3)
CNN shapes:  (480044, 1, 88, 50) (41034, 1, 88, 50) (39295, 1, 88, 50)
Labels train/dev/test: (480044,) (41034,) (39295,)


In [13]:

np.save(BASE_DIR/'cnn_train_data.npy',  X_train_c)
np.save(BASE_DIR/'cnn_train_labels.npy', y_train)
np.save(BASE_DIR/'cnn_dev_data.npy',    X_dev_c)
np.save(BASE_DIR/'cnn_dev_labels.npy',   y_dev)
np.save(BASE_DIR/'cnn_test_data.npy',   X_test_c)
np.save(BASE_DIR/'cnn_test_labels.npy',  y_test)
print("✅ CNN feature files written.")

✅ CNN feature files written.


In [ ]:
# Hybrid Dataset & DataLoaders
import torch
from torch.utils.data import Dataset, DataLoader

class HybridDataset(Dataset):
    def __init__(self, lstm_data, cnn_data, labels):
        self.lstm = torch.tensor(lstm_data, dtype=torch.float32)
        self.cnn  = torch.tensor(cnn_data,  dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.lstm[idx], self.cnn[idx], self.labels[idx]

# Build datasets
train_ds = HybridDataset(X_train_l, X_train_c, y_train)
dev_ds   = HybridDataset(X_dev_l,   X_dev_c,   y_dev)
test_ds  = HybridDataset(X_test_l,  X_test_c,  y_test)

# Create loaders
batch_size = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
dev_loader   = DataLoader(dev_ds,   batch_size=batch_size)
test_loader  = DataLoader(test_ds,  batch_size=batch_size)


lstm_b, cnn_b, y_b = next(iter(train_loader))
print("LSTM batch shape:", lstm_b.shape)   
print("CNN batch shape: ", cnn_b.shape)    
print("Label batch shape:", y_b.shape)    

LSTM batch shape: torch.Size([32, 50, 3])
CNN batch shape:  torch.Size([32, 1, 88, 50])
Label batch shape: torch.Size([32])
